In [1]:
import pandas as pd
import matplotlib as plt
import re
import numpy as np

##### reading in data 

In [2]:
yr2023 = pd.read_csv('C:/Users/madel/Documents/CRIME/ftwayne2023.csv')
yr2023.head()                     

,Incident number,Incident nature,Area location code,Incident address,When reported,Occurred after,Occurred before,City,Offense code,Offense code.1,Offense code.2,Disposition
0,23F000001,113 SHOTS FIRED,F11A5,1945 GRISWOLD DR,1/1/2023 0:01,1/1/2023 0:00,1/1/2023 0:00,FORT WAYNE,113,,,UNF
1,23F000003,113 SHOTS FIRED,F41B5,WINTER ST & E LEWIS ST,1/1/2023 0:02,1/1/2023 0:02,1/1/2023 0:02,FORT WAYNE,113,,,INA
2,23F000004,"43,62",F1643,2737 W WASHINGTON CENTER RD LOT 364,1/1/2023 0:06,1/1/2023 0:05,1/1/2023 0:05,FORT WAYNE,DFCL,CDIS,,R
3,23F000005,113 SHOTS FIRED,F42A5,305 E DEWALD ST,1/1/2023 0:01,1/1/2023 0:00,1/1/2023 0:00,FORT WAYNE,113,,,INA
4,23F000006,48 SUSPICIOUS,F32D5,3410 S CALHOUN ST,1/1/2023 0:10,1/1/2023 0:08,1/1/2023 0:08,FORT WAYNE,48,,,INA


In [3]:
yr2023["Incident nature"].str.extract(r'(\d*)')

,0
0,113
1,113
2,43
3,113
4,48
...,...
154473,111
154474,111
154475,111
154476,48


##### determining violent/non violent crimes

In [4]:
yr2023["Incident nature #"] = yr2023["Incident nature"].str.extract(r'(\d*)')
yr2023["Incident nature"] = yr2023["Incident nature"].str.extract(r'([a-z|\s]+)', flags = re.IGNORECASE)
yr2023["Incident nature"].str.strip('\s')
yr2023.head()

,Incident number,Incident nature,Area location code,Incident address,When reported,Occurred after,Occurred before,City,Offense code,Offense code.1,Offense code.2,Disposition,Incident nature #
0,23F000001,SHOTS FIRED,F11A5,1945 GRISWOLD DR,1/1/2023 0:01,1/1/2023 0:00,1/1/2023 0:00,FORT WAYNE,113,,,UNF,113
1,23F000003,SHOTS FIRED,F41B5,WINTER ST & E LEWIS ST,1/1/2023 0:02,1/1/2023 0:02,1/1/2023 0:02,FORT WAYNE,113,,,INA,113
2,23F000004,,F1643,2737 W WASHINGTON CENTER RD LOT 364,1/1/2023 0:06,1/1/2023 0:05,1/1/2023 0:05,FORT WAYNE,DFCL,CDIS,,R,43
3,23F000005,SHOTS FIRED,F42A5,305 E DEWALD ST,1/1/2023 0:01,1/1/2023 0:00,1/1/2023 0:00,FORT WAYNE,113,,,INA,113
4,23F000006,SUSPICIOUS,F32D5,3410 S CALHOUN ST,1/1/2023 0:10,1/1/2023 0:08,1/1/2023 0:08,FORT WAYNE,48,,,INA,48


In [5]:
incident = list(yr2023["Incident nature"].unique())
incident = [str(x).strip(' ') for x in incident]

In [14]:
#filename = 'C:/Users/madel/Documents/CRIME/incident_list.txt'
#with open(filename, 'w') as outfile:
#    for item in incident:
#        # write each item on a new line
#        outfile.write("%s\n" % item)

In [7]:
#non violent list: SUSPICIOUS, FIRE, VAND VEH, CUTTING, T SUICIDE THR, RAPE, FIGHT, THEFT, ARMED ROB,
#BATTERY, A SUICIDE ATT, MAN DOWN, BOMB THREAT, SHOOTING, PARTY ARMED, I INTRUSION, N CHILD NEGLE,
#MOLESTING, 
#idk list: " ", nan,
#PLACES (why????): GRABILL, WOODBURN, MONROEVILLE

#### nan values

In [8]:
yr2023.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [9]:
yr2023.isna().sum()

Incident number            0
Incident nature        17791
Area location code       844
Incident address         202
When reported              0
Occurred after             0
Occurred before            0
City                     286
Offense code            2547
Offense code.1        145360
Offense code.2        151993
Disposition              284
Incident nature #       2807
dtype: int64

In [13]:
yr2023.columns

Index(['Incident number', 'Incident nature', 'Area location code',
       'Incident address', 'When reported', 'Occurred after',
       'Occurred before', 'City', 'Offense code', 'Offense code.1',
       'Offense code.2', 'Disposition', 'Incident nature #'],
      dtype='object')

#### visualizations

In [3]:
from geopy.geocoders import Nominatim

In [ ]:
geolocator = Nominatim(user_agent="")
location = geolocator.geocode("175 5th Avenue NYC")
yr2023["lat"] = location.latitude

#plotly.express